# 5.2 训练一个LLM

​		在本节中，我们最终实现了用于预训练 LLM 的代码，即我们的 GPTModel。为此，我们专注于一个简单的训练循环，如图 5.11 所示，以保持代码简洁易读。但是，有兴趣的读者可以在附录 D，向训练循环添加花里胡哨中了解更高级的技术，包括学习速率预热、余弦退火和梯度削波。

图 5.11 在 PyTorch 中训练深度神经网络的典型训练循环由几个步骤组成，在训练集中的批次上迭代多个时期。在每个循环中，我们计算每个训练集批次的损失以确定损失梯度，我们用它来更新模型权重，以便将训练集损失降至最低。

![image-20240422143154243](../img/image-5-11.png)

​		图 5.11 中的流程图描述了一个典型的 PyTorch 神经网络训练工作流程，我们用它来训练 LLM。它概述了八个步骤，从迭代每个时期开始，处理批处理，重置和计算梯度，更新权重，最后是监控步骤，如打印损失和生成文本样本。如果您对使用 PyTorch 训练深度神经网络比较陌生，并且不熟悉其中任何一个步骤，请考虑阅读附录 A，PyTorch 简介中的 A.5 至 A.8 部分。

​		在代码中，我们可以通过以下train_model_simple函数实现此训练流程：

**Listing 5.3 预训练 LLM 的主要功能**

In [ ]:
def train_model_simple(model, train_loader, val_loader, optimizer, device, num_epochs,
eval_freq, eval_iter, start_context):
    
	train_losses, val_losses, track_tokens_seen = [], [], [] #A
	tokens_seen, global_step = 0, -1
	for epoch in range(num_epochs): #B
        model.train()
        for input_batch, target_batch in train_loader:
            optimizer.zero_grad() #C
            loss = calc_loss_batch(input_batch, target_batch, model, device)
            loss.backward() #D
            optimizer.step() #E
            tokens_seen += input_batch.numel()
            global_step += 1
            if global_step % eval_freq == 0: #F
                train_loss, val_loss = evaluate_model(
                model, train_loader, val_loader, device, eval_iter)
                train_losses.append(train_loss)
                val_losses.append(val_loss)
                track_tokens_seen.append(tokens_seen)
                print(f"Ep {epoch+1} (Step {global_step:06d}): "
                f"Train loss {train_loss:.3f}, Val loss {val_loss:.3f}")
        generate_and_print_sample( #G
        	model, train_loader.dataset.tokenizer, device, start_context
        )
	return train_losses, val_losses, track_tokens_seen

​		请注意，我们刚刚创建的 train_model_simple 函数使用了两个尚未定义的函数：evaluate_model 和 generate_and_print_sample。

​		evaluate_model 函数对应于图 5.11 中的步骤 7。它会在每次模型更新后打印训练和验证集损失，以便我们可以评估训练是否改进了模型。

​		更具体地说，evaluate_model函数计算训练和验证集的损失，同时确保模型处于评估模式，在计算训练和验证集的损失时禁用梯度跟踪和辍学：

In [ ]:
def evaluate_model(model, train_loader, val_loader, device, eval_iter):
    model.eval() #A
    with torch.no_grad(): #B
        train_loss = calc_loss_loader(train_loader, model, device, num_batches=eval_iter)
        val_loss = calc_loss_loader(val_loader, model, device, num_batches=eval_iter)
    model.train()
    return train_loss, val_loss

​		与 evaluate_model 类似，generate_and_print_sample 函数是一个方便函数，我们用它来跟踪模型在训练过程中是否改进。具体而言，generate_and_print_sample 函数将文本片段 （start_context） 作为输入，将其转换为令牌 ID，并将其提供给 LLM，以使用我们之前使用的 generate_text_simple 函数生成文本示例：

In [ ]:
def generate_and_print_sample(model, tokenizer, device, start_context):
    model.eval()
    context_size = model.pos_emb.weight.shape[0]
    encoded = text_to_token_ids(start_context, tokenizer).to(device)
    with torch.no_grad():
        token_ids = generate_text_simple(
            model=model, idx=encoded,
            max_new_tokens=50, context_size=context_size
        )
        decoded_text = token_ids_to_text(token_ids, tokenizer)
        print(decoded_text.replace("\n", " ")) # Compact print format
    model.train()

​		虽然 evaluate_model 函数为我们提供了模型训练进度的数字估计，但这个generate_and_print_sample文本函数提供了模型生成的具体文本示例，用于判断其在训练期间的能力。

**AdamW**

​		Adam 优化器是训练深度神经网络的热门选择。但是，在我们的训练循环中，我们选择了 AdamW 优化器。AdamW 是 Adam 的一个变体，它改进了权重衰减方法，旨在通过惩罚更大的权重来最大限度地降低模型复杂性并防止过度拟合。这种调整使 AdamW 能够实现更有效的正则化和更好的泛化，因此经常用于 LLM 的训练。

​		让我们通过使用 AdamW 优化器和我们之前定义的 train_model_simple 函数训练 10 个 epoch 的 GPTModel 实例来了解这一切。

In [ ]:
torch.manual_seed(123)
model = GPTModel(GPT_CONFIG_124M)
model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.0004, weight_decay=0.1) #A
num_epochs = 10
train_losses, val_losses, tokens_seen = train_model_simple(
    model, train_loader, val_loader, optimizer, device,
    num_epochs=num_epochs, eval_freq=5, eval_iter=1,
    start_context="Every effort moves you"
)

​		执行 training_model_simple 功能将启动训练过程，在 MacBook Air 或类似笔记本电脑上大约需要 5 分钟才能完成。在此执行过程中打印的输出如下：

In [ ]:
Ep 1 (Step 000000): Train loss 9.781, Val loss 9.933
Ep 1 (Step 000005): Train loss 8.111, Val loss 8.339
Every effort moves you,,,,,,,,,,,,.
Ep 2 (Step 000010): Train loss 6.661, Val loss 7.048
Ep 2 (Step 000015): Train loss 5.961, Val loss 6.616
Every effort moves you, and, and, and, and, and, and, and, and, and, and, and, and, and, and, and, and, and, and, and, and, and, and,, and, and,
[...] #A
Ep 9 (Step 000080): Train loss 0.541, Val loss 6.393
Every effort moves you?" "Yes--quite insensible to the irony. She wanted him vindicated--and by me!" He laughed again, and threw back the window-curtains, I had the donkey. "There were days when I
Ep 10 (Step 000085): Train loss 0.391, Val loss 6.452
Every effort moves you know," was one of the axioms he laid down

​		正如我们所看到的，根据训练期间打印的结果，训练损失急剧改善，从值 9.558 开始，收敛到 0.762。该模型的语言技能有了很大的提高。在开始时，模型只能将逗号附加到开始上下文中（“Every effort moves you,,,,,,,,,,,,”）或重复单词“and”。在训练结束时，它可以生成语法正确的文本。

​		与训练集损失类似，我们可以看到验证损失从高处开始 （9.856），并在训练期间减少。但是，它永远不会变得像训练集损失那么小，并且在第 10 个纪元之后保持在 6.372。

​		在更详细地讨论验证损失之前，让我们创建一个简单的图，并排显示训练集和验证集损失：

In [ ]:
import matplotlib.pyplot as plt
def plot_losses(epochs_seen, tokens_seen, train_losses, val_losses):
    fig, ax1 = plt.subplots(figsize=(5, 3))
    ax1.plot(epochs_seen, train_losses, label="Training loss")
    ax1.plot(epochs_seen, val_losses, linestyle="-.", label="Validation loss")
    ax1.set_xlabel("Epochs")
    ax1.set_ylabel("Loss")
    ax1.legend(loc="upper right")
    ax2 = ax1.twiny() #A
    ax2.plot(tokens_seen, train_losses, alpha=0) #B
    ax2.set_xlabel("Tokens seen")
    fig.tight_layout()
    plt.show()
epochs_tensor = torch.linspace(0, num_epochs, len(train_losses))
plot_losses(epochs_tensor, tokens_seen, train_losses, val_losses)

​		得到的训练和验证损失图如图 5.12 所示。

图 5.12 在训练开始时，我们观察到训练集和验证集的损失都急剧减少，这表明模型正在学习。但是，训练集损失在第二个时期之后继续减少，而验证损失停滞不前。这表明模型仍在学习，但它与第 2 期之后的训练集过度拟合。

![image-20240422144030197](..\img\image-5-12.png)

​		如图 5.12 所示，在第一个时期，训练和验证损失都开始改善。然而，损失开始分化超过第二个时代。这种背离以及验证损失远大于训练损失的事实表明模型对训练数据过度拟合。我们可以通过搜索生成的文本片段来确认模型逐字记住了训练数据，例如“The Verdict”文本文件中的“对讽刺非常不敏感”。

​		这种记忆是意料之中的，因为我们正在使用一个非常非常小的训练数据集，并为多个时期训练模型。通常，通常只针对一个时期在更大的数据集上训练模型。

​		如前所述，感兴趣的读者可以尝试在古腾堡计划的 60,000 本公共领域书籍上训练模型，其中不会发生这种过度拟合;详见附录B。

​		在下一节中，如图 5.13 所示，我们将探讨 LLM 采用的采样方法来减轻记忆效应，从而生成更新颖的文本。

图 5.13 我们的模型在实现训练函数后可以生成连贯的文本。但是，它经常逐字记住训练集中的段落。以下部分介绍生成更多样化输出文本的策略。

![image-20240422144152449](../img/image-5-13.png)

​		如图 5.13 所示，下一节将介绍 LLM 的文本生成策略，以减少训练数据记忆并提高 LLM 生成文本的原创性，然后我们介绍权重加载以及保存和加载来自 OpenAI 的 GPT 模型的预训练权重。